In [ ]:
#Instalar si se usa por primera vez
!pip install pandas openpyxl python-Levenshtein
!pip install pandas Jinja2
!pip install reportlab
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install gensim
!python -m spacy download es_core_news_sm #No borrar!!! Si no no jala :(

In [ ]:
import pandas as pd
import re
import pandas as pd
import os
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
import datetime
import pandas as pd
import re
import spacy
import nltk
import csv
import gensim
import numpy as np
from collections import Counter
from fuzzywuzzy import fuzz
import io
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.graphics.shapes import Circle, Drawing
from reportlab.graphics.renderPDF import draw
import os
import pandas as pd

In [ ]:
#Cosas en español
nltk.download('wordnet')
nltk.download('omw')
nlp = spacy.load("es_core_news_sm")
#COnectores discursivos
def cargar_conectores(path):
    con_disc = set()
    with open(path, newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            con_disc.update(row)
    return con_disc

con_disc = cargar_conectores("/content/drive/MyDrive/conectores dfiscursivos1.csv")
#Más cosas en español
nltk.download('stopwords')
stopword_es = nltk.corpus.stopwords.words('spanish')

In [ ]:
def limpiar(texto):
    if pd.isna(texto):
        return ""

    texto = texto.lower()
    texto = re.sub(r"(100)\b\d+", "", texto)
    texto = re.sub(r"(-1)\b\d+", "", texto)
    texto = re.sub(r"[^\w\s/*-]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    doc = nlp(texto)
    tokens = [
        token.text for token in doc
        if not token.is_stop
        and token.lemma_ not in con_disc
        and token.text not in stopword_es
    ]

    return " ".join(tokens)

def limpiar_tv(texto):
    texto = texto.lower()
    texto = texto.strip()
    matches = re.findall(r"\(([^()]*)\)", texto)
    if matches:
        for match in matches:
            if '/' in match:
                return match
    return texto
def cociente(text):
    match = re.search(r"(.+?)\s*/\s*(.+)", text)
    if match:
        num = match.group(1).strip()
        den = match.group(2).strip()
        similarity_ratio = fuzz.ratio(num, den)
        return similarity_ratio >= 50
    return False

def comparacion(metodo_p, tipo_r):
    if tipo_r == '(A/B)*100':
        pattern = r".*\s*[xX\*]?\s*100"
        if (re.search(pattern, metodo_p) or re.search(r".*\/.*\\b(programado|planificado|programada|planificada|multiplicado por 100|x100|x 100|X100|X 100|\*100|\* 100)\\b", metodo_p)) and not re.search(r".*\/.*\\b(solicitadas|recibidas|requeridas|demandadas|gestionadas|tramitadas)\\b", metodo_p):
            return 'Sí'
    elif tipo_r == 'Tasa de variación':
        pattern = r'\s*1(?:\s*[xX\*]?\s*100)?'
        if cociente(limpiar_tv(metodo_p)) and (re.search(pattern, metodo_p) or re.search(r".*\/.*\\b(anterior|pasado|año pasado|mes pasado|año anterior|ciclo anterior|ciclo pasado|año t-1|año t - 1| año t- 1|año t -1|t-1|-1*100|-1 * 100|-1 x 100| -1 X 100|t - 1|t- 1|t -1|n-1|n - 1|n -1|n- 1|t1|t 1)\\b",metodo_p)):
                return 'Sí'
    elif tipo_r == 'Absoluto':
        pattern = r".*\s*/\s*.*"
        if not re.search(pattern, metodo_p):
                return 'Sí'
    return 'No'
#Método para comparar los componentes
def comp_met(comp_1, comp_2):
  similitud = fuzz.ratio(comp_1, comp_2)
  if similitud >= 50:
    return "Sí"
  else:
    return "No"


def comparar_otros(row):
    if row['TIPO_RELATIVO_METODO_COMPARACION'] == 'No' or row['TIPO_RELATIVO_METODO_COMPARACION'] == 'Otro':
        metodo_p = row['METODO_PROCESADO']
        if (re.search( r".*\s*[xX\*]?\s*100", metodo_p) or re.search(r".*\/.*\\b(programado|planificado|programada|planificada|multiplicado por 100|x100|x 100|X100|X 100|\*100|\* 100)\\b", metodo_p))  and not (re.search(r".*\/.*\b(solicitadas|recibidas|requeridas|demandadas|gestionadas|tramitadas)\b", metodo_p) and re.search(r'-\s*1.*?[xX\*]?\s*100', metodo_p)):
            return '(A/B)*100'
        elif cociente(metodo_p) and (re.search(r'\s*1.*?[xX\*]?\s*100', metodo_p) or re.search(r".*\/.*\\b(anterior|pasado|año pasado|mes pasado|año anterior|ciclo anterior|ciclo pasado|año t-1|año t - 1| año t- 1|año t -1||t-1|t - 1|t- 1|t -1|n-1|n - 1|n -1|n- 1|t1|t 1)\\b",metodo_p)):
            return 'Tasa de variación'
        elif not (re.search(r".*\s*/\s*.*" , metodo_p) or (p in metodo_p for p in ['cociente', '/', 'entre', 'dividido', 'respecto', 'dividiendo'])):
            return 'Absoluto'
    return 'No'

def comparacion_indicador(info):
    columnas = ["TIPO_RELATIVO", "METODO_CALCULO_INDICADOR", "NOMBRE_INDICADOR", "DEFINICION_INDICADOR"]
    if not all(col in info.columns for col in columnas):
        raise ValueError(f"Faltan las siguientes columnas: {', '.join(columnas)}")
    info_copy = info.copy()

    #Comparación entre el método de cálculo y el tipo relativo
    info_copy['METODO_PROCESADO'] = info_copy.apply(
        lambda row: row['METODO_CALCULO_INDICADOR'] if pd.isna(row['TIPO_RELATIVO']) or row['TIPO_RELATIVO'] == "Tasa de variación" else limpiar(row['METODO_CALCULO_INDICADOR']),
        axis=1
    )
    info_copy['TIPO_RELATIVO_METODO_COMPARACION'] = info_copy.apply(
        lambda row: comparacion(row['METODO_PROCESADO'], row['TIPO_RELATIVO']),
        axis=1
    )

    #Limpiar las demás columnas para evaluar
    info_copy['NOMBRE_PROCESADO'] = info_copy.apply(
        lambda row: limpiar(row['NOMBRE_INDICADOR']),
        axis=1
    )
    info_copy['DEFINICION_PROCESADO'] = info_copy.apply(
        lambda row: limpiar(row['DEFINICION_INDICADOR']),
        axis=1
    )

    info_copy['OTRA_COINCIDENCIA'] = None

    #COMPARACIÓN
    otra_coin = (info_copy['TIPO_RELATIVO_METODO_COMPARACION'] == 'No') & (~info_copy['TIPO_RELATIVO_METODO_COMPARACION'].isna())
    info_copy.loc[otra_coin, 'OTRA_COINCIDENCIA'] = info_copy.loc[otra_coin].apply(comparar_otros, axis=1)

    otra_coin = (info_copy['TIPO_RELATIVO'] == 'Otro')
    info_copy.loc[otra_coin, 'OTRA_COINCIDENCIA'] = info_copy.loc[otra_coin].apply(comparar_otros, axis=1)

    #Comparación nombre y definición
    info_copy['NOMBRE_DEFINICION_COMPARACION'] = info_copy.apply(
        lambda row: comp_met(str(row['NOMBRE_PROCESADO']), str(row['DEFINICION_PROCESADO'])),
        axis=1
    )

    #Comparación definición y método de cálculo
    info_copy['DEFINICION_METODO_COMPARACION'] = info_copy.apply(
        lambda row: comp_met(str(row['DEFINICION_PROCESADO']), str(row['METODO_PROCESADO'])),
        axis=1
    )

    return info_copy[['METODO_CALCULO_INDICADOR', 'TIPO_RELATIVO_METODO_COMPARACION', 'OTRA_COINCIDENCIA', 'NOMBRE_DEFINICION_COMPARACION', 'DEFINICION_METODO_COMPARACION']]

In [ ]:
def validar_metas_asc(info):
    resultado = {
        "vacios": "",
        "umbrales": "",
        "metas": "",
        "mensaje_final": "",
        "patron_metas": ""
    }

    #Ver si está vacío
    if info.empty:
        resultado["mensaje_final"] = "No hay datos para validar."
        return resultado

    fila = info.iloc[0]

    #pasar las metas a datos numéricos
    metas_cols = [f"META{i}" for i in range(1, 13)]
    metas = [pd.to_numeric(fila.get(col), errors='coerce') for col in metas_cols]

    #Pasar la ifno que usaremos a numerico
    ciclo_base = pd.to_numeric(fila.get('CICLO_LINEA_BASE'), errors='coerce')
    linea_base = pd.to_numeric(fila.get('LINEA_BASE'), errors='coerce')
    umbral_verde = pd.to_numeric(fila.get('UMBRAL_VERDE'), errors='coerce')
    umbral_rojo = pd.to_numeric(fila.get('UMBRAL_ROJO'), errors='coerce')
    meta_anual = pd.to_numeric(fila.get("META12"), errors='coerce')

    no_12 = pd.isna(meta_anual)
    frecuencia = fila.get("FRECUENCIA_MEDICION")

    #1. Guardar la info referente a las metas que está vacía
    resultado["vacios"] = (
        f"{'Frecuencia no definida; ' if pd.isna(frecuencia) else ''}"
        f"{'El ciclo de la línea base no está definido; ' if pd.isna(ciclo_base) else ''}"
        f"{'La línea base no está definida; ' if pd.isna(linea_base) else ''}"
        f"{'Umbral verde no definido; ' if pd.isna(umbral_verde) else ''}"
        f"{'Umbral rojo no definido; ' if pd.isna(umbral_rojo) else ''}"
        f"{'Meta anual no definida; ' if no_12 else ''}"
    )

    #2. Validar los umbrales
    if (not pd.isna(umbral_rojo) and
        not pd.isna(umbral_verde) and
        not pd.isna(meta_anual)):

        if umbral_verde > umbral_rojo:
            if meta_anual >= linea_base:
                resultado["umbrales"] = "La meta anual coincide con la línea base coinciden con el sentido; "
            else:
                resultado["umbrales"] = "La meta anual debe ser mayor o igual a la línea base; "
        else:
            resultado["umbrales"] = "El umbral verde debe ser mayor al rojo; "
    else:
        resultado["umbrales"] = ""

    #Aquí se generan los errores según la frecuencia y si hay metas que no coinciden con el sentido registrado. Ver si la meta anual (12) es mayor o igual que la línea base
    def generar_errores(indices, frecuencia_reg):
        mensajes = []
        for i in reversed(range(1, len(indices))):
            idx_actual = indices[i] - 1
            idx_anterior = indices[i-1] - 1
            meta_actual = metas[idx_actual]
            meta_anterior = metas[idx_anterior]
            if not pd.isna(meta_actual) and not pd.isna(meta_anterior) and meta_actual < meta_anterior:
                mensajes.append(f"META {indices[i]} debe ser mayor o igual que META {indices[i-1]}; ")
        if len(indices) > 0:
            first_meta_idx = indices[0] - 1
            if not pd.isna(metas[first_meta_idx]) and not pd.isna(linea_base) and metas[first_meta_idx] < linea_base:
                mensajes.append(f"META {indices[0]} debe ser mayor o igual que la línea base; ")
        return "".join(mensajes)

    #Aquí se ve si es acumulado, constante o mal determinado
    def determinar_patron(indices):
        valid_metas = [metas[i-1] for i in indices if not pd.isna(metas[i-1])]
        if len(valid_metas) < 1:
            return "No hay suficientes metas para saber si es acumulado o constante"
        diferencias = [valid_metas[i] - valid_metas[i-1] for i in range(1, len(valid_metas))]

        if all(d > 0 for d in diferencias):
            return "Acumulado"
        if all(d == 0 for d in diferencias):
            return "Constante"
        if all(d < 0 for d in diferencias):
            return "Sentido mal determinado"

        return "No se pudo determinar el sentido"

    #Aquí se ve se ve si las metas son cocnsistentes para lkas frecuencias bianuales y trianual
    def frecuencia_especial(metas_esperadas):
        metas_no_na = [m for m in metas if not pd.isna(m)]

        if len(metas_no_na) != metas_esperadas:
            return {"error": f"Se requieren exactamente {metas_esperadas} metas para frecuencia {frecuencia} (encontradas: {len(metas_no_na)}); "}
        errores = []
        patron = None
        if metas_esperadas == 2:
            meta1, meta2 = metas_no_na[0], metas_no_na[1]

            if meta1 > meta2:
                patron = "Sentido mal determinado"
                errores.append("META1 debe ser menor o igual que la META2; ")
            elif meta1 == meta2:
                patron = "Constante"
            else:
                patron = "Acumulado"

            if not pd.isna(linea_base) and meta1 < linea_base:
                errores.append("META1 debe ser mayor o igual que la línea base; ")
        elif metas_esperadas == 3:
            meta1, meta2, meta3 = metas_no_na[0], metas_no_na[1], metas_no_na[2]

            if meta1 > meta2 or meta2 > meta3:
                if meta1 > meta2 and meta2 > meta3:
                    patron = "Sentido mal determinado"
                elif meta1 == meta2 and meta2 == meta3:
                    patron = "Constante"
                elif meta1 < meta2 and meta2 < meta3:
                    patron = "Acumulado"
                else:
                    patron = "No se pudo determinar el sentido"

                if meta1 > meta2:
                    errores.append("META1 debe ser menor o igual que la META2; ")
                if meta2 > meta3:
                    errores.append("META2 debe ser menor o igual que la META3; ")
            else:
                patron = "Acumulado"

            if not pd.isna(linea_base) and meta1 < linea_base:
                errores.append("META1 debe ser mayor o igual que la línea base; ")

        if not errores:
            errores.append(f"Las metas coinciden con el sentido {patron.lower()} y línea base; ")

        return {"error": "".join(errores), "patron": patron}

    #Aquí se usan las funciones definidas
    if frecuencia == "Bianual":
        validacion = frecuencia_especial(2)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    elif frecuencia == "Trianual":
        validacion = frecuencia_especial(3)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    else:
      #Aquí es poara obtener el índice de donde están las metas correspondientes a la frecuecnia
        indices_map = {
            "Mensual": list(range(1, 13)),
            "Bimestral": [2, 4, 6, 8, 10, 12],
            "Trimestral": [3, 6, 9, 12],
            "Cuatrimestral": [4, 8, 12],
            "Semestral": [6, 12],
            "Anual": [12],
            "Bienal": [12],
            "Trienal": [12],
            "Quinquenal": [12],
            "Sexenal": [12]
        }
        #Aquí es para saber cuantas debe haber vacías según la frecuencia registrada

        umbral_vacias_map = {
            "Mensual": 0,
            "Bimestral": 6,
            "Trimestral": 8,
            "Cuatrimestral": 9,
            "Semestral": 10,
            "Anual": 11,
            "Bienal": 11,
            "Trienal": 11,
            "Quinquenal": 11,
            "Sexenal": 11
        }

        indices = indices_map.get(frecuencia)
        umbral_vacias = umbral_vacias_map.get(frecuencia, 12)

        if indices is not None:
            err = generar_errores(indices, frecuencia)
            current_metas_for_freq = [metas[i-1] for i in indices]
            if sum(1 for m in current_metas_for_freq if pd.isna(m)) > umbral_vacias:
                resultado["metas"] = "Sin suficientes metas registradas; "
            elif not err:
                resultado["metas"] = "Las metas coinciden con el sentido y línea base; "
            else:
                resultado["metas"] = err

            patron = determinar_patron(indices)
            resultado["patron_metas"] = patron if patron is not None else ""

            if no_12 and frecuencia in ["Anual", "Bienal", "Trienal", "Quinquenal", "Sexenal"]:
                resultado["patron_metas"] = "No es posible determinar el sentido porque falta la meta anual"
        else:
            resultado["metas"] = "Frecuencia de medición no reconocida; "

    #para los mensajes sobre la validación de metas
    resultado["mensaje_final"] = (
        resultado["vacios"] +
        resultado["umbrales"] +
        resultado["metas"] +
        (resultado["patron_metas"] if resultado["patron_metas"] else "")
    )

    return resultado

In [ ]:
def validar_metas_desc(info):
    resultado = {
        "vacios": "",
        "umbrales": "",
        "metas": "",
        "mensaje_final": "",
        "patron_metas": ""
    }

    if info.empty:
        resultado["mensaje_final"] = "No hay datos para validar."
        return resultado

    fila = info.iloc[0]

    metas_cols = [f"META{i}" for i in range(1, 13)]
    metas = [pd.to_numeric(fila.get(col), errors='coerce') for col in metas_cols]
    ciclo_base = pd.to_numeric(fila.get('CICLO_LINEA_BASE'), errors='coerce')
    linea_base = pd.to_numeric(fila.get('LINEA_BASE'), errors='coerce')
    umbral_verde = pd.to_numeric(fila.get('UMBRAL_VERDE'), errors='coerce')
    umbral_rojo = pd.to_numeric(fila.get('UMBRAL_ROJO'), errors='coerce')
    meta_anual = pd.to_numeric(fila.get("META12"), errors='coerce')

    no_12 = pd.isna(meta_anual)
    frecuencia = fila.get("FRECUENCIA_MEDICION")

    resultado["vacios"] = (
        f"{'Frecuencia no definida; ' if pd.isna(frecuencia) else ''}"
        f"{'El ciclo de la línea base no está definido; ' if pd.isna(ciclo_base) else ''}"
        f"{'La línea base no está definida; ' if pd.isna(linea_base) else ''}"
        f"{'Umbral verde no definido; ' if pd.isna(umbral_verde) else ''}"
        f"{'Umbral rojo no definido; ' if pd.isna(umbral_rojo) else ''}"
        f"{'Meta anual no definida; ' if no_12 else ''}"
    )

    if (not pd.isna(umbral_rojo) and
        not pd.isna(umbral_verde) and
        not pd.isna(meta_anual)):
        if umbral_verde < umbral_rojo:
            if meta_anual <= linea_base:
                resultado["umbrales"] = "Los umbrales y la línea base coinciden con el sentido; "
            else:
                resultado["umbrales"] = "La meta anual debe ser menor o igual a la línea base; "
        else:
            resultado["umbrales"] = "El umbral verde debe ser menor al rojo; "
    else:
        resultado["umbrales"] = ""

    def generar_errores(indices, frecuencia_reg):
        mensajes = []

        for i in reversed(range(1, len(indices))):
            idx_actual = indices[i] - 1
            idx_anterior = indices[i-1] - 1

            meta_actual = metas[idx_actual]
            meta_anterior = metas[idx_anterior]

            if not pd.isna(meta_actual) and not pd.isna(meta_anterior) and meta_actual > meta_anterior:
                mensajes.append(f"META {indices[i]} debe ser menor o igual que META {indices[i-1]}; ")

        if len(indices) > 0:
            first_meta_idx = indices[0] - 1
            if not pd.isna(metas[first_meta_idx]) and not pd.isna(linea_base) and metas[first_meta_idx] > linea_base:
                mensajes.append(f"META {indices[0]} debe ser menor o igual que la línea base; ")

        return "".join(mensajes)

    def determinar_patron(indices):
        valid_metas = [metas[i-1] for i in indices if not pd.isna(metas[i-1])]
        if len(valid_metas) < 1:
            return None
        diferencias = [valid_metas[i] - valid_metas[i-1] for i in range(1, len(valid_metas))]
        if all(d < 0 for d in diferencias):
            return "Acumulado"
        if all(d == 0 for d in diferencias):
            return "Constante"
        if all(d > 0 for d in diferencias):
            return "Sentido mal determinado"
        return "No se pudo determinar el sentido"


    def frecuencia_especial(metas_esperadas):
        metas_no_na = [m for m in metas if not pd.isna(m)]
        if len(metas_no_na) != metas_esperadas:
            return {"error": f"Se requieren exactamente {metas_esperadas} metas para frecuencia {frecuencia} (encontradas: {len(metas_no_na)}); "}
        errores = []
        patron = None
        if metas_esperadas == 2:
            meta1, meta2 = metas_no_na[0], metas_no_na[1]
            if meta1 < meta2:
                patron = "Sentido mal determinado"
                errores.append("META1 debe ser mayor o igual que la META2; ")
            elif meta1 == meta2:
                patron = "Constante"
            else:
                patron = "Acumulado"
            if not pd.isna(linea_base) and meta1 > linea_base:
                errores.append("META1 debe ser menor o igual que la línea base; ")

        elif metas_esperadas == 3:
            meta1, meta2, meta3 = metas_no_na[0], metas_no_na[1], metas_no_na[2]

            if meta1 < meta2 or meta2 < meta3:
                if meta1 < meta2 and meta2 < meta3:
                    patron = "Sentido mal determinado"
                elif meta1 == meta2 and meta2 == meta3:
                    patron = "Constante"
                elif meta1 > meta2 and meta2 > meta3:
                    patron = "Acumulado"
                else:
                    patron = "No se pudo determinar el sentido"
                if meta1 < meta2:
                    errores.append("META1 debe ser mayor o igual que la META2; ")
                if meta2 < meta3:
                    errores.append("META2 debe ser mayor o igual que la META3; ")
            else:
                patron = "Acumulado"
            if not pd.isna(linea_base) and meta1 > linea_base:
                errores.append("META1 debe ser menor o igual que la línea base; ")
        if not errores:
            errores.append(f"Las metas coinciden con el sentido {patron.lower()} y línea base; ")
        return {"error": "".join(errores), "patron": patron}

    if frecuencia == "Bianual":
        validacion = frecuencia_especial(2)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    elif frecuencia == "Trianual":
        validacion = frecuencia_especial(3)
        resultado["metas"] = validacion["error"]
        resultado["patron_metas"] = validacion["patron"]
    else:
        indices_map = {
            "Mensual": list(range(1, 13)),
            "Bimestral": [2, 4, 6, 8, 10, 12],
            "Trimestral": [3, 6, 9, 12],
            "Cuatrimestral": [4, 8, 12],
            "Semestral": [6, 12],
            "Anual": [12],
            "Bienal": [12],
            "Trienal": [12],
            "Quinquenal": [12],
            "Sexenal": [12]
        }
        umbral_vacias_map = {
            "Mensual": 0,
            "Bimestral": 6,
            "Trimestral": 8,
            "Cuatrimestral": 9,
            "Semestral": 10,
            "Anual": 11,
            "Bienal": 11,
            "Trienal": 11,
            "Quinquenal": 11,
            "Sexenal": 11
        }
        indices = indices_map.get(frecuencia)
        umbral_vacias = umbral_vacias_map.get(frecuencia, 12)

        if indices is not None:
            err = generar_errores(indices, frecuencia)
            current_metas_for_freq = [metas[i-1] for i in indices]
            if sum(1 for m in current_metas_for_freq if pd.isna(m)) > umbral_vacias:
                resultado["metas"] = "Sin suficientes metas registradas; "
            elif not err:
                resultado["metas"] = "Las metas coinciden con el sentido y línea base; "
            else:
                resultado["metas"] = err
            patron = determinar_patron(indices)
            resultado["patron_metas"] = patron if patron is not None else ""

            if no_12 and frecuencia in ["Anual", "Bienal", "Trienal", "Quinquenal", "Sexenal"]:
                resultado["patron_metas"] = "No es posible determinar el sentido porque falta la meta anual"
        else:
            resultado["metas"] = "Frecuencia de medición no reconocida; "

    resultado["mensaje_final"] = (
        resultado["vacios"] +
        resultado["umbrales"] +
        resultado["metas"] +
        (resultado["patron_metas"] if resultado["patron_metas"] else "")
    )

    return resultado

In [ ]:
def FICHA_INDICADOR(indicador, ruta):
    #Cargar info
    try:
        info = pd.read_excel(ruta, sheet_name="Registros", dtype=str)
    except FileNotFoundError:
        return {
            "info": None,
            "validacion": {"mensaje": "Archivo no encontrado"},
            "corresponde": None,
            "sentido": None,
            "comparacion_metodo_res": None
        }
    except Exception as e:
        return {
            "info": None,
            "validacion": {"mensaje": f"Error al leer el archivo: {e}"},
            "corresponde": None,
            "sentido": None,
            "comparacion_metodo_res": None
        }

    #Para que todas las metas se llamen igual
    if "META_REG1" in info.columns and "META1" not in info.columns:
        info.rename(columns={"META_REG1": "META1"}, inplace=True)

    #Columnas que se necesitan
    initial_cols = [
        "CICLO", "RAMO", "PP", "DESC_PROYECTO", "TIPO_NIVEL", "LLAVE_INDICADOR",
        "NOMBRE_INDICADOR", "DEFINICION_INDICADOR", "METODO_CALCULO_INDICADOR",
        "TIPO_RELATIVO", "FRECUENCIA_MEDICION", "DESC_SENTIDO", "UMBRAL_VERDE",
        "UMBRAL_ROJO", "LINEA_BASE", "CICLO_LINEA_BASE"
    ]
    meta_cols_pattern = r"META\d+"
    meta_cols = [col for col in info.columns if re.match(meta_cols_pattern, col)]

    cols_to_select = initial_cols + sorted(meta_cols)

    info = info[info["LLAVE_INDICADOR"] == str(indicador)][cols_to_select]

    #Si no hay info
    if info.empty:
        return {
            "info": None,
            "validacion": {"mensaje": "Indicador no encontrado"},
            "corresponde": None,
            "sentido": None,
            "comparacion_metodo_res": None
        }

    fila = info.iloc[0]

    #Ver si hay info
    no_info_parts = []
    if pd.isna(fila.get("NOMBRE_INDICADOR")): no_info_parts.append("Nombre del indicador no definido; ")
    if pd.isna(fila.get("DEFINICION_INDICADOR")): no_info_parts.append("Definición del indicador no definido; ")
    if pd.isna(fila.get("METODO_CALCULO_INDICADOR")): no_info_parts.append("Método de cálculo del indicador no definido; ")
    no_info = "".join(no_info_parts)

    #Para que coincida el tipo
    char_cols = [
        "PP", "DESC_PROYECTO", "TIPO_NIVEL", "NOMBRE_INDICADOR", "DEFINICION_INDICADOR",
        "METODO_CALCULO_INDICADOR", "TIPO_RELATIVO", "FRECUENCIA_MEDICION", "DESC_SENTIDO"
    ]

    for col in info.columns:
        if col in char_cols:
            info[col] = info[col].astype(str).replace('nan', None)
        else:
            info[col] = pd.to_numeric(info[col], errors='coerce')

    fila = info.iloc[0]

    if pd.isna(info['TIPO_RELATIVO'].iloc[0]):
      info['TIPO_RELATIVO'].iloc[0] = "Absoluto"

    #Si tiene frecuencia rara, ver si le toca o no revisión 
    corresponde = "La frecuencia registrada no es bienal, trienal, quinquenal o sexenal"
    if not pd.isna(fila.get('CICLO')) and not pd.isna(fila.get('CICLO_LINEA_BASE')) and not pd.isna(fila.get('FRECUENCIA_MEDICION')):
        ciclo_diff = fila['CICLO'] - fila['CICLO_LINEA_BASE']
        if fila['FRECUENCIA_MEDICION'] == "Bienal":
            corresponde = "Corresponde" if ciclo_diff % 2 == 0 else "No corresponde"
        elif fila['FRECUENCIA_MEDICION'] == "Trienal":
            corresponde = "Corresponde" if ciclo_diff % 3 == 0 else "No corresponde"
        elif fila['FRECUENCIA_MEDICION'] == "Quinquenal":
            corresponde = "Corresponde" if ciclo_diff % 5 == 0 else "No corresponde"
        elif fila['FRECUENCIA_MEDICION'] == "Sexenal":
            corresponde = "Corresponde" if ciclo_diff % 6 == 0 else "No corresponde"

    #Si tiene sentido
    sentido = ""
    if corresponde != "No corresponde":
        if pd.isna(fila.get("DESC_SENTIDO")):
            sentido = "Sentido no definido"
        else:
            sentido = f"Sentido definido: {fila['DESC_SENTIDO']}"
    else:
        sentido = "No aplica (no corresponde medición)"

    #validar las metas según el snetido, si el sentido es vacío, entonces se evalúa como ascendente
    validacion = {"mensaje": ""}
    if corresponde == "Corresponde" or corresponde == "La frecuencia registrada no es bienal, trienal, quinquenal o sexenal":
        if pd.isna(fila.get("DESC_SENTIDO")) or fila["DESC_SENTIDO"] == "Ascendente":
            validacion = validar_metas_asc(info.copy())
        elif fila["DESC_SENTIDO"] == "Descendente":
            validacion = validar_metas_desc(info.copy())
        else:
            validacion = {"mensaje": "Error en el sentido registrado"}
    else:
        validacion = {"mensaje": "No corresponde medición"}

    #Comparar método de cálculo
    comparacion_metodo_res = {"mensaje": ""}
    if corresponde == "Corresponde" or corresponde == "La frecuencia registrada no es bienal, trienal, quinquenal o sexenal":
        comparacion_metodo_res = comparacion_indicador(info[['NOMBRE_INDICADOR', 'DEFINICION_INDICADOR','TIPO_RELATIVO', 'METODO_CALCULO_INDICADOR']].copy())
    else:
        comparacion_metodo_res = {"mensaje": "No corresponde medición"}

    return {
        "info": info,
        "no_info": no_info,
        "validacion": validacion,
        "corresponde": corresponde,
        "sentido": sentido,
        "comparacion_metodo_res": comparacion_metodo_res
    }

In [ ]:
ficha_resultado = FICHA_INDICADOR(llave, "/path/archivo.xlsx") #Aquí se espera que la llave sea del tipo 123456

In [ ]:
print(ficha_resultado)

In [ ]:
def ajustar_imagen(ruta, ancho_max, alto_max):
    img = Image(ruta)
    ancho_img, alto_img = img.drawWidth, img.drawHeight

    width_scale = ancho_max / ancho_img
    height_scale = alto_max / alto_img

    scale = max(width_scale, height_scale)

    img.drawWidth = ancho_img * scale
    img.drawHeight = alto_img * scale
    return img

In [ ]:
def recom_obs(info_df, validacion_list, comparacion_metodo_df):
    salida = []

    # SENTIDO
    sentido_definido = not info_df.empty and info_df.get('DESC_SENTIDO').iloc[0] != ""
    sentido_congruente = validacion_list.get('metas') == "Los umbrales y la línea base coinciden con el sentido; " and validacion_list.get('metas') != "Sentido mal determinado"

    if sentido_definido:
        salida.append({"criterio": "Sentido definido", "score": 0.5, "recomendacion": "El indicador tiene un sentido definido.", "observacion": "Información del sentido del indicador registrada."})
    else:
        salida.append({"criterio": "Sentido definido", "score": 0, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "No se registró sentido"})

    if sentido_congruente:
        salida.append({"criterio": "Congruencia del sentido", "score": 0.5, "recomendacion": "El sentido del indicador es adecuado.", "observacion": ""})
    elif (sentido_definido != sentido_congruente):
         salida.append({"criterio": "Congruencia del sentido", "score": 0.25, "recomendacion": "Se recomienda revisar la congruencia del sentido del indicador con su línea base o con las metas programadas", "observacion": "El sentido del indicador corresponde con un elemento."})
    else:
        salida.append({"criterio": "Congruencia del sentido", "score": 0, "recomendacion": "Se recomienda revisar la congruencia del sentido del indicador con su línea base y con las metas programadas", "observacion": "El sentido del indicador no es congruente con la línea base ni con las metas programadas"})

    # LÍNEA BASE
    linea_base_definida = validacion_list.get('vacios') != 'El ciclo de la línea base no está definido; ' and validacion_list.get('vacios') != 'La línea base no está definida; '
    if linea_base_definida:
        salida.append({"criterio": "Línea base definida", "score": 0.5, "recomendacion": "La línea base y el ciclo de la línea base están definidos.", "observacion": "Línea base y ciclo de la línea base definidos."})
    elif (validacion_list.get('vacios') == 'El ciclo de la línea base no está definido; ') != (validacion_list.get('vacios') == 'La línea base no está definida; '):
        salida.append({"criterio": "Línea base definida", "score": 1/4, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Línea base o ciclo línea de la línea base no definido"})
    else:
        salida.append({"criterio": "Línea base definida", "score": 0, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "No se regitró línea base ni su ciclo."})

    linea_base_congruente = validacion_list.get('umbrales') == "La meta anual coincide con la línea base coinciden con el sentido; "
    indices = validacion_list.get('indices')
    if linea_base_congruente:
        salida.append({"criterio": "Congruencia de la línea base", "score": 0.5, "recomendacion": "La línea base es congruente con las metas y el sentido del indicador.", "observacion": ""})
    elif (validacion_list.get('umbrales') == "La meta anual debe ser menor o igual a la línea base; " or validacion_list.get('umbrales') == "La meta anual debe ser mayor o igual a la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser mayor o igual que la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser menor o igual que la línea base; "):
         salida.append({"criterio": "Congruencia de la línea base", "score": 1/4, "recomendacion": "Se recomienda ajustar la línea base para que sea congruente con las metas y sentido del indicador.", "observacion": "La meta anual o la primer meta no son consistentes con el sentido del indicador."})
    else:
        salida.append({"criterio": "Congruencia de la línea base", "score": 0, "recomendacion": "Se recomienda establecer una línea base sólida, con fuente y año claros.", "observacion": "La meta anual y la primer meta no son consistentes con el sentido del indicador."})


    # SEMAFORIZACIÓN
    umbrales_definidos = validacion_list.get('vacios') != "Umbral rojo no definido; " and validacion_list.get('vacios') != "Umbral verde no definido; "
    if umbrales_definidos:
        salida.append({"criterio": "Umbrales definidos", "score": 1/2, "recomendacion": "", "observacion": "Están definidos ambos umbrales"})
    elif (validacion_list.get('vacios') == "Umbral rojo no definido; ") != (validacion_list.get('vacios') == "Umbral verde no definido; "):
        salida.append({"criterio": "Umbrales definidos", "score": 1/4, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Uno de los umbrales no está definido."})
    else:
        salida.append({"criterio": "Umbrales definidos", "score": 0, "recomendacion": "Se recomienda definir ambos umbrales.", "observacion": "Umbrales no definidos."})

    umbrales_validos = validacion_list.get('umbrales') == ""
    if umbrales_validos:
        salida.append({"criterio": "Validación umbrales", "score": 1/2, "recomendacion": "Se recomienda validar periódicamente la pertinencia de los rangos definidos.", "observacion": "Los umbrales coinciden con el sentido definido."})
    else:
        salida.append({"criterio": "Validación umbrales", "score": 0, "recomendacion": "Se recomienda definir o rediseñar los parámetros de semaforización con base técnica, metas realistas y criterios claros.", "observacion": "Los umbrales no coinciden con el sentido definido."})

    # FRECUENCIA
    frecuencia_definida = validacion_list.get('vacios') != "Frecuencia no definida; "
    if frecuencia_definida:
        salida.append({"criterio": "Frecuencia definida", "score": 1, "recomendacion": "Frecuencia definida.", "observacion": "Frecuencia de medición del indicador definida."})
    else:
        salida.append({"criterio": "Frecuencia definida", "score": 0, "recomendacion": "Se recomienda definir la frecuencia de medición del indicador.", "observacion": "Frecuencia de medición del indicador no definida."})

    # META ANUAL / PRIMER META
    meta_anual_definida = validacion_list.get('vacios') != 'Meta anual no definida; '
    if meta_anual_definida:
        salida.append({"criterio": "Meta anual definida", "score": 0.5, "recomendacion": "Meta anual definida.", "observacion": "Meta anual definida."})
    else:
        salida.append({"criterio": "Meta anual definida", "score": 0, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Meta anual no definida."})

    meta_anual_consistente = not ((validacion_list.get('umbrales') == "La meta anual debe ser menor o igual a la línea base; ") or (validacion_list.get('umbrales') == "La meta anual debe ser mayor o igual a la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser mayor o igual que la línea base; ") or (indices and validacion_list.get('mensajes') == f"META {indices[0]} debe ser menor o igual que la línea base; "))

    if meta_anual_consistente:
        salida.append({"criterio": "Consistencia de la meta anual", "score": 0.5, "recomendacion": "La meta anual y la primer meta son consistentes con el sentido del indicador.", "observacion": "La meta anual y la primer meta son consistentes con el sentido del indicador."})
    else:
        salida.append({"criterio": "Consistencia de la meta anual", "score": 0, "recomendacion": "La meta anual o la primer meta no son consistentes con el sentido del indicador.", "observacion": "La meta anual o la primer meta no son consistentes con el sentido del indicador."})

    ##METAS CORRESPONDIENTES A LA FRECUENCIA REGSISTRADA
    metas_frecuencia_definidas = validacion_list.get('metas') != "Sin suficientes metas registradas; "
    if metas_frecuencia_definidas:
        salida.append({"criterio": "Metas correspondientes a la frecuencia definida", "score": 0.5, "recomendacion": "Están definidas las metas correspondientes a la frecuencia.", "observacion": "Metas definidas correspondientes a la frecuencia definida."})
    else:
        salida.append({"criterio": "Metas correspondientes a la frecuencia definida", "score": 0, "recomendacion": "Se recomienda revisar las metas registradas o la frecuencia definida.", "observacion": "No se registraron metas correspondientes a la frecuencia definida."})

    metas_frecuencia_consistentes = validacion_list.get('patron_metas') != "Sentido mal determiando"
    if metas_frecuencia_consistentes:
        salida.append({"criterio": "Consistencia de las metas con el sentido", "score": 0.5, "recomendacion": "Las metas son consistentes con el sentido del indicador.", "observacion": "Las metas son consistentes con el sentido del indicador."})
    else:
        salida.append({"criterio": "Consistencia de las metas con el sentido", "score": 0, "recomendacion": "Se recomienda revisar y modificar metas o revisar el sentido y ajustarlo si es necesario.", "observacion": "Las metas no son consistentes con el sentido del indicador."})


    # CONGRUENCIA DE NOMBRE, MÉTODO DE CÁLCULO Y DEFINICIÓN DEL INDICADOR
    parametros_definidos = validacion_list.get('no_info') != "Nombre del indicador no definido; " and validacion_list.get('no_info') != "Definición del indicador no definida; " and validacion_list.get('no_info') != "Método de cálculo del indicador no definido; "
    if parametros_definidos:
        salida.append({"criterio": "Parámetros definidos", "score": 1/3, "recomendacion": "El nombre, definición y método de cálculo del indicador están definidos.", "observacion": "Nombre, definición y método de cálculo del indicador definidos."})
    elif (validacion_list.get('no_info') == "Nombre del indicador no definido; ") != (validacion_list.get('vacios') == "Definición del indicador no definida; ") != (validacion_list.get('vacios') == "Método de cálculo del indicador no definido; "):
         salida.append({"criterio": "Parámetros definidos", "score": 1/6, "recomendacion": "Se recomienda revisar la información registrada.", "observacion": "Nombre, definición o método de cálculo del indicador no definidos."})
    else:
        salida.append({"criterio": "Parámetros definidos", "score": 0, "recomendacion": "Se recomienda definir el nombre, definición y método de cálculo del indicador.", "observacion": "Nombre, definición y método de cálculo del indicador no definidos."})

    nombre_definicion_congruentes = not comparacion_metodo_df.empty and comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION').iloc[0] == "Sí"
    if nombre_definicion_congruentes:
        salida.append({"criterio": "Nombre y definición congruentes", "score": 1/3, "recomendacion": "El nombre y la definición del indicador son congruentes.", "observacion": "Nombre y definición del indicador congruentes."})
    else:
        salida.append({"criterio": "Nombre y definición congruentes", "score": 0, "recomendacion": "Se recomienda revisar la congruencia entre el nombre y la definición del indicador.", "observacion": "Nombre y definición del indicador no son congruentes."})

    definicion_metodo_congruentes = not comparacion_metodo_df.empty and comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION').iloc[0] == "Sí"
    if definicion_metodo_congruentes:
        salida.append({"criterio": "Definición y método de cálculo congruente", "score": 1/3, "recomendacion": "La definición y el método de cálculo del indicador son congruentes.", "observacion": "Definición y método de cálculo del indicador congruentes."})
    else:
        salida.append({"criterio": "Definición y método de cálculo congruente", "score": 0, "recomendacion": "Se recomienda revisar la congruencia entre la definición y el método de cálculo del indicador.", "observacion": "Definición y método de cálculo del indicador no son congruentes."})

    # COMPARACIÓN MÉTODO DE CÁLCULO Y TIPO RELATIVO
    tipo_relativo_metodo_congruente = not comparacion_metodo_df.empty and comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION').iloc[0] == "Sí"
    if tipo_relativo_metodo_congruente:
        salida.append({"criterio": "Tipo relativo y método de cálculo congruentes", "score": 1, "recomendacion": "El tipo relativo y el método de cálculo del indicador son congruentes.", "observacion": "Tipo relativo y método de cálculo del indicador congruentes."})
    else:
        salida.append({"criterio": "Tipo relativo y método de cálculo congruentes", "score": 0, "recomendacion": "Se recomienda revisar la congruencia entre el tipo relativo y el método de cálculo del indicador.", "observacion": "Tipo relativo y método de cálculo del indicador no son congruentes."})

    return salida

In [ ]:
def generar_ficha_pdf(ficha_resultado,
                     titulo="Índice de Seguimiento de los Indicadores de Desempeño",
                     nombre_archivo="ficha_indicadores.pdf",
                     ruta=None,
                     imagen_izquierda_path=None,
                     imagen_derecha_path=None):

    if ruta is None:
        ruta = os.getcwd()

    os.makedirs(ruta, exist_ok=True)

    output_path = os.path.join(ruta, nombre_archivo)
    doc = SimpleDocTemplate(output_path, pagesize=letter)
    styles = getSampleStyleSheet()
    #Definir estilos para la tabla
    styles.add(ParagraphStyle(name='WrappedTableText', parent=styles['Normal'], leading=12))

    story = []

    #Imágenes en el encabezado
    header_elements = []
    MAX_IMG_ANCHO = 1.0 * inch
    MAX_IMG_ALTO = 0.75 * inch

    if imagen_izquierda_path and os.path.exists(imagen_izquierda_path):
        img_left = ajustar_imagen(imagen_izquierda_path, MAX_IMG_ANCHO, MAX_IMG_ALTO)
        header_elements.append(img_left)
    else:
        header_elements.append(Paragraph("", styles['Normal']))

    if imagen_derecha_path and os.path.exists(imagen_derecha_path):
        img_right = ajustar_imagen(imagen_derecha_path, MAX_IMG_ANCHO, MAX_IMG_ALTO)
        header_elements.append(img_right)
    else:
        header_elements.append(Paragraph("", styles['Normal']))

    if header_elements:
        header_table = Table([header_elements], colWidths=[letter[0]/2, letter[0]/2])
        header_table.setStyle(TableStyle([
            ('VALIGN', (0,0), (-1,-1), 'TOP'),
            ('ALIGN', (0,0), (0,0), 'LEFT'),
            ('ALIGN', (1,0), (1,0), 'RIGHT'),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING', (0,0), (-1,-1), 0),
            ('TOPPADDING', (0,0), (-1,-1), 0),
            ('BOTTOMPADDING', (0,0), (-1,-1), 0),
        ]))
        story.append(header_table)
        story.append(Spacer(1, 0.2*inch))

    #Para el título
    story.append(Paragraph(titulo, styles['Title']))
    story.append(Spacer(1, 0.2*inch))

    #Información de la ficha de resultado
    info_df = ficha_resultado.get('info', pd.DataFrame())
    no_info_str = ficha_resultado.get('no_info', '')
    validacion_list = ficha_resultado.get('validacion', {})
    corresponde_str = ficha_resultado.get('corresponde', '')
    sentido_str = ficha_resultado.get('sentido', '')
    comparacion_metodo_df = ficha_resultado.get('comparacion_metodo_res', {})

    #INFORMACIÓN BÁSICA
    info_data = [["Seguimiento de los Indicadores"]]
    if not info_df.empty:
        info_data.append(['Ciclo', Paragraph(str(info_df.get('CICLO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Ramo', Paragraph(str(info_df.get('RAMO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Clave', Paragraph(str(info_df.get('PP', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Nombre Pp', Paragraph(str(info_df.get('DESC_PROYECTO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Nivel', Paragraph(str(info_df.get('TIPO_NIVEL', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Indicador', Paragraph(str(info_df.get('KA_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Nombre del indicador', Paragraph(str(info_df.get('NOMBRE_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Definición del indicador', Paragraph(str(info_df.get('DEFINICION_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Método de cálculo', Paragraph(str(info_df.get('METODO_CALCULO_INDICADOR', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Tipo relativo', Paragraph(str(info_df.get('TIPO_RELATIVO', '').iloc[0]), styles['WrappedTableText'])])
        info_data.append(['Sentido', Paragraph(str(info_df.get('DESC_SENTIDO', '').iloc[0]), styles['WrappedTableText'])])
    else:
        info_data.append([Paragraph('No hay datos disponibles para este indicador.', styles['WrappedTableText']), ''])


    info_table = Table(info_data, colWidths=[2*inch, 5*inch]) 
    info_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#2A4277")), 
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.HexColor("#EDAB25")),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('ALIGN', (0, 1), (0, -1), 'LEFT'), 
        ('ALIGN', (1, 1), (-1, -1), 'LEFT'), 
        ('VALIGN', (0, 0), (-1, -1), 'TOP'), 
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,0), 6),
        ('RIGHTPADDING', (0,0), (-1,0), 6),
        ('TOPPADDING', (0,0), (-1,0), 6),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(info_table)
    story.append(Spacer(1, 0.2*inch))

    #VALIDACIÓN DE METAS
    story.append(Spacer(1, 0.1*inch))

    validacion_data = [["Validación de Metas"]]
    validacion_data.append(['Campos vacíos', Paragraph(validacion_list.get('vacios', ''), styles['WrappedTableText'])])
    validacion_data.append(['Umbrales', Paragraph(validacion_list.get('umbrales', ''), styles['WrappedTableText'])])
    validacion_data.append(['Metas', Paragraph(validacion_list.get('metas', ''), styles['WrappedTableText'])])
    validacion_data.append(['Sentido determinado', Paragraph(validacion_list.get('patron_metas', ''), styles['WrappedTableText'])])

    validacion_table = Table(validacion_data, colWidths=[2*inch, 5*inch]) 
    validacion_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')), 
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('ALIGN', (0, 1), (0, -1), 'LEFT'), 
        ('ALIGN', (1, 1), (-1, -1), 'LEFT'), 
         ('VALIGN', (0, 0), (-1, -1), 'TOP'), 
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(validacion_table)
    story.append(Spacer(1, 0.2*inch))

    #COMPARACIÓN DE MÉTODO DE CÁLCULO, DEFINICIÓN, ETC
    comparacion_data = [["Comparación de Métodos"]]

    comparacion_data.append([Paragraph('Comparación entre nombre y definición', styles['WrappedTableText']), Paragraph(str(comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION', '').iloc[0]) if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION'), pd.Series) else comparacion_metodo_df.get('NOMBRE_DEFINICION_COMPARACION', ''), styles['WrappedTableText'])])
    comparacion_data.append([Paragraph('Comparación entre definición y método de cálculo', styles['WrappedTableText']), Paragraph(str(comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION', '').iloc[0]) if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION'), pd.Series) else comparacion_metodo_df.get('DEFINICION_METODO_COMPARACION', ''), styles['WrappedTableText'])])
    comparacion_data.append([Paragraph('Comparación entre tipo relativo y método de cálculo registrado', styles['WrappedTableText']), Paragraph(str(comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION', '').iloc[0]) if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION'), pd.Series) else comparacion_metodo_df.get('TIPO_RELATIVO_METODO_COMPARACION', ''), styles['WrappedTableText'])])
    comparacion_data.append([Paragraph('Comparación con otro tipo relativo', styles['WrappedTableText']), comparacion_metodo_df.get('OTRA_COINCIDENCIA', '').iloc[0] if not comparacion_metodo_df.empty and isinstance(comparacion_metodo_df.get('OTRA_COINCIDENCIA'), pd.Series) else comparacion_metodo_df.get('OTRA_COINCIDENCIA', '')])

    comparacion_table = Table(comparacion_data, colWidths=[2*inch, 5*inch])
    comparacion_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')), 
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('ALIGN', (0, 1), (0, -1), 'LEFT'), 
        ('ALIGN', (1, 1), (-1, -1), 'LEFT'), 
         ('VALIGN', (0, 0), (-1, -1), 'TOP'), 
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(comparacion_table)
    story.append(Spacer(1, 0.2*inch))

    #OBSERVACIONES Y PUNTAJE
    validation_results = recom_obs(info_df, validacion_list, comparacion_metodo_df)
    score_data = [["Criterios", "Puntaje", ""]]

    for result in validation_results:
        criterio = result["criterio"]
        score_value = result["score"]  
        recomendacion = result["recomendacion"]
        observacion = result["observacion"]

        diameter = 0.2 * inch
        radius = diameter / 2.0
        circle_drawing = Drawing(diameter, diameter)
        circle_color = colors.grey 

        if score_value > 1/4:
            circle_color = colors.HexColor('#4CAF50')  
        elif score_value > 0:
            circle_color = colors.HexColor('#FFEB3B') 
        else:
            circle_color = colors.HexColor('#F44336') 

        circle = Circle(radius, radius, radius)
        circle.fillColor = circle_color
        circle.strokeColor = circle_color
        circle_drawing.add(circle)

        #Para mostrar el cirterio, la observación y el semáforo correspondiente
        display_text = f"<b>{criterio}:</b> {observacion}"
        score_data.append([Paragraph(display_text, styles['WrappedTableText']), f"{score_value:.2f}", circle_drawing])


    score_table = Table(score_data, colWidths=[5*inch, 1*inch, 1*inch])
    score_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
    ]))
    story.append(score_table)
    story.append(Spacer(1, 0.2*inch))

    #RECOMENDACION
    oportunidad_data = [["Recomendaciones"]]
    for result in validation_results:
        recomendacion = result["recomendacion"]
        display_text = f"<b>{result['criterio']}:</b> {recomendacion}"
        oportunidad_data.append([Paragraph(display_text, styles['WrappedTableText'])])

    oportunidad_table = Table(oportunidad_data, colWidths=[7*inch]) 
    oportunidad_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2A4277')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('FONTSIZE', (0, 0), (-1, 0), 16),
        ('ALIGN', (0, 0), (-1, 0), 'CENTRE'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 6),
        ('BOTTOMPADDING', (0,0), (-1,-1), 12),
        ('SPAN',(0, 0), (-1, 0)),
        ('ALIGN', (0, 1), (-1, -1), 'LEFT'), 
        ('VALIGN', (0, 1), (-1, -1), 'TOP'),
        ('SPAN',(0, 0), (-1, 0)),
    ]))
    story.append(oportunidad_table)
    story.append(Spacer(1, 0.2*inch))


    try:
        doc.build(story)
        print(f"Ficha generada en: {output_path}")
    except Exception as e:
        print(f"Error generating PDF with ReportLab: {e}")

In [ ]:
generar_ficha_pdf(ficha_resultado,
                      titulo="Ficha de Indicador",
                      nombre_archivo="ficha_indicadores1.pdf",
                      ruta="/content",
                      imagen_izquierda_path="/path/imagen1.jpg",
                      imagen_derecha_path="/path/imagen2.jpg")